In [1]:
import spacy
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from gap_statistic import OptimalK

ModuleNotFoundError: No module named 'spacy'

In [ ]:
# Nội dung văn bản
content = """Hôm nay chúng ta họp để bàn về kế hoạch triển khai dự án quý tới. 
Đầu tiên, nhóm kỹ thuật sẽ cần hoàn thành việc nâng cấp hệ thống vào cuối tháng này. 
Điều này rất quan trọng để đảm bảo không ảnh hưởng đến tiến độ chung. 
Tiếp theo, nhóm kinh doanh đã liên hệ với một số đối tác và họ cần phản hồi sớm để kịp ký kết hợp đồng. 
Ngoài ra, vấn đề ngân sách cũng cần được xem xét vì có một số khoản chi chưa được phê duyệt. 
Chúng ta cần đưa ra quyết định sớm để đảm bảo nguồn lực đầy đủ. 
Cuối cùng, tôi đề xuất tổ chức một cuộc họp nhỏ vào tuần sau để kiểm tra tiến độ từng nhóm trước khi có báo cáo chính thức. 
Nếu không còn ý kiến gì thêm, chúng ta kết thúc cuộc họp tại đây.!"""

In [ ]:
# Tiền xử lý văn bản
contents_parsed = content.lower()  # Chuyển tất cả sang chữ thường
contents_parsed = contents_parsed.replace('\n', '. ')  # Thay đổi ký tự xuống dòng thành dấu chấm câu
contents_parsed = contents_parsed.strip()  # Loại bỏ khoảng trắng thừa

In [ ]:
# In kết quả sau khi xử lý
contents_parsed

'hôm nay chúng ta họp để bàn về kế hoạch triển khai dự án quý tới. . đầu tiên, nhóm kỹ thuật sẽ cần hoàn thành việc nâng cấp hệ thống vào cuối tháng này. . điều này rất quan trọng để đảm bảo không ảnh hưởng đến tiến độ chung. . tiếp theo, nhóm kinh doanh đã liên hệ với một số đối tác và họ cần phản hồi sớm để kịp ký kết hợp đồng. . ngoài ra, vấn đề ngân sách cũng cần được xem xét vì có một số khoản chi chưa được phê duyệt. . chúng ta cần đưa ra quyết định sớm để đảm bảo nguồn lực đầy đủ. . cuối cùng, tôi đề xuất tổ chức một cuộc họp nhỏ vào tuần sau để kiểm tra tiến độ từng nhóm trước khi có báo cáo chính thức. . nếu không còn ý kiến gì thêm, chúng ta kết thúc cuộc họp tại đây.!'

In [5]:
import spacy
from pyvi import ViTokenizer

ModuleNotFoundError: No module named 'spacy'

In [33]:
# Khởi tạo mô hình spaCy trống cho tiếng Việt
nlp = spacy.blank("vi")

# Thêm Sentencizer vào pipeline để xác định ranh giới câu
nlp.add_pipe("sentencizer")

# Phân tích văn bản
doc = nlp(contents_parsed)

# Lấy danh sách câu
sentences = [sent.text for sent in doc.sents]

# Đếm số câu trong danh sách sentences
num_sentences = len(sentences)
print(f"Số câu trong văn bản: {num_sentences}")

print(sentences)


Số câu trong văn bản: 8
['hôm nay chúng ta họp để bàn về kế hoạch triển khai dự án quý tới. .', 'đầu tiên, nhóm kỹ thuật sẽ cần hoàn thành việc nâng cấp hệ thống vào cuối tháng này. .', 'điều này rất quan trọng để đảm bảo không ảnh hưởng đến tiến độ chung. .', 'tiếp theo, nhóm kinh doanh đã liên hệ với một số đối tác và họ cần phản hồi sớm để kịp ký kết hợp đồng. .', 'ngoài ra, vấn đề ngân sách cũng cần được xem xét vì có một số khoản chi chưa được phê duyệt. .', 'chúng ta cần đưa ra quyết định sớm để đảm bảo nguồn lực đầy đủ. .', 'cuối cùng, tôi đề xuất tổ chức một cuộc họp nhỏ vào tuần sau để kiểm tra tiến độ từng nhóm trước khi có báo cáo chính thức. .', 'nếu không còn ý kiến gì thêm, chúng ta kết thúc cuộc họp tại đây.!']


In [34]:
from transformers import AutoTokenizer, AutoModel
import torch

In [35]:
# Load PhoBERT
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModel.from_pretrained("vinai/phobert-base")

In [36]:
# Chuyển từng câu thành vector
X = []
for sentence in sentences:
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    sentence_vec = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()  # Lấy mean pooling
    
    X.append(sentence_vec)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [37]:

print("Vector của câu đầu tiên:", X[0])

Vector của câu đầu tiên: [-1.44541338e-01  1.96281075e-01 -1.91772208e-02 -1.29348323e-01
  6.10941015e-02  1.55328780e-01  2.22732246e-01 -2.14256644e-01
 -3.41902561e-02 -1.36493921e-01  2.61065699e-02  6.49976134e-02
 -1.00475445e-01  3.56359407e-02 -6.07962906e-02  1.08285651e-01
  2.31777150e-02  1.87922083e-02 -1.64273866e-02 -2.82295167e-01
 -2.28835702e-01 -2.52002835e-01  3.48294675e-01  2.43375093e-01
 -3.48876536e-01 -1.26819000e-01  3.58913362e-01  9.40350816e-02
  2.37719715e-01 -1.69980794e-01 -3.92086618e-02 -1.06320575e-01
 -1.91202506e-01  4.29077595e-02  5.86768270e-01  2.63390303e-01
  1.38172358e-01 -6.79946169e-02  6.46356493e-02  3.44668001e-01
 -2.66886558e-02  1.90824002e-01  6.72454014e-02  1.09201089e-01
  1.25981674e-01  7.11825415e-02  1.90880746e-01 -2.80792683e-01
  2.90255547e-02  2.41848379e-01  1.03770018e-01 -1.24559857e-01
 -1.29398420e-01  6.43837631e-01  1.02766842e-01 -2.81626612e-01
  1.25570246e-03  3.50813009e-02  4.85363528e-02 -6.22064061e-02


In [38]:
# import fasttext

# # Load mô hình fastText
# fasttext_model = fasttext.load_model("cc.vi.300.bin")

# # Vector của câu đầu tiên với fastText
# fasttext_vector = fasttext_model.get_sentence_vector(sentences[0])

# # Tính khoảng cách Cosine giữa hai vector
# distance = cosine(X[0], fasttext_vector)
# print("Khoảng cách Cosine giữa vector PhoBERT và fastText:", distance)


In [39]:
import numpy as np

# Chuyển X thành NumPy array nếu nó là list
if isinstance(X, list):
    X = np.array(X)

print(f"Kích thước X: {X.shape}")  # Kiểm tra X có đúng dạng (số câu, 768) hay không


Kích thước X: (8, 768)


In [40]:
from gap_statistic import OptimalK
import numpy as np

optimalK = OptimalK(parallel_backend='none')

# Đảm bảo số cụm tối đa không lớn hơn số câu có trong dữ liệu
K_range = np.arange(2, min(10, X.shape[0]))  # Giới hạn số cụm hợp lý

K_optimal = optimalK(X, cluster_array=K_range)

print(f"Số cụm K tối ưu (Gap Statistic): {K_optimal}")


Số cụm K tối ưu (Gap Statistic): 5


In [41]:
from sklearn.cluster import KMeans

# Chọn số cụm K (ví dụ: 3 cụm)
K = K_optimal
kmeans = KMeans(n_clusters=K, random_state=42, n_init=10)

# Thực hiện phân cụm
kmeans.fit(X)

# Lấy nhãn cụm của từng câu
labels = kmeans.labels_


In [42]:
import numpy as np
from sklearn.metrics import pairwise_distances_argmin_min

# Lấy số cụm từ K-Means
n_clusters = K_optimal

# Tìm câu gần trung tâm cụm nhất
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, X)

# Tính thứ tự xuất hiện trung bình của từng cụm
avg = []
for j in range(n_clusters):
    idx = np.where(kmeans.labels_ == j)[0]  # Lấy chỉ mục các câu thuộc cụm j
    avg.append(np.mean(idx))  # Tính thứ tự xuất hiện trung bình

# Sắp xếp cụm theo thứ tự xuất hiện trung bình
ordering = sorted(range(n_clusters), key=lambda k: avg[k])

# Xây dựng đoạn văn bản tóm tắt
summary = ' '.join([sentences[closest[idx]] for idx in ordering])

# Hiển thị đoạn tóm tắt
print("Văn bản tóm tắt:")
print(summary)


Văn bản tóm tắt:
hôm nay chúng ta họp để bàn về kế hoạch triển khai dự án quý tới. . điều này rất quan trọng để đảm bảo không ảnh hưởng đến tiến độ chung. . cuối cùng, tôi đề xuất tổ chức một cuộc họp nhỏ vào tuần sau để kiểm tra tiến độ từng nhóm trước khi có báo cáo chính thức. . ngoài ra, vấn đề ngân sách cũng cần được xem xét vì có một số khoản chi chưa được phê duyệt. . chúng ta cần đưa ra quyết định sớm để đảm bảo nguồn lực đầy đủ. .
